In [1]:
import spotpy
import numpy as np
import pandas as pd
import subprocess
import shutil
import os

In [2]:
class spotpy_setup():
    # the __init__ runs just once at the begining of the optimization
    def __init__(self):
        # define the parameters for the model
        self.params = [spotpy.parameter.Uniform('gw1',0.0001,0.4),
                    spotpy.parameter.Uniform('gw2',0.0001,1.),
                    spotpy.parameter.Uniform('m',0.01,100.),
                    spotpy.parameter.Uniform('ksat',0.0001,150.),
                    spotpy.parameter.Uniform('crd',2.,10.),
                    spotpy.parameter.Uniform('trd',0.1,2.),
                    spotpy.parameter.Uniform('po',0.1,30.),
                    spotpy.parameter.Uniform('pa',0.1,30.)]
        
        self.obspath = '/RHESSys/Como/obs/como_q_obs.pcl'
        self.outpath = '/RHESSys/Como/out/opt'
        
        self.start_date = pd.datetime(2006,10,1) # simulation spinup start (RHESSys format)
        self.end_date = pd.datetime(2009,10,5) # simulation end date (RHESSys format)
        self.a_start_date = '2007-10-1' # analysis start date, pandas format
        self.a_end_date = '2009-09-30' # analysis end date, pandas format
        
        # read in the observations
        obs = pd.read_pickle(self.obspath)[self.a_start_date:self.a_end_date]
        obs['datetime'] = obs.index
        evaldates = obs.datetime # pull out the observation dates
        obs = obs.discharge.as_matrix() # pull out the observations
        # generate the clean indexer, 1 = nan, 0 = data
        clean_indexer = np.isnan(obs) # generate an indexer for the nan values
        obs = obs[clean_indexer==0] # remove the nan values
        evaldates = evaldates[clean_indexer==0] # remove the dates with nans
        self.obs = obs # add the cropped and clean obs to the self object
        self.evaldates = evaldates # add the cropped and clean dates to the self object
        self.clean_indexer = clean_indexer # add the index values of the non-nan obs to the object


    def parameters(self):
        # generate the parameters
        return spotpy.parameter.generate(self.params)


    def simulation(self,vector):
        
        gw1 = vector[0]
        gw2 = vector[1]
        m = vector[2]
        ksat = vector[3]
        crd = vector[4]
        trd = vector[5]
        po = vector[6]
        pa = vector[7]
        
        #create the output directory
        #diridx = '%s%s%s%s%s%s%s%s'%(gw1,gw2,m,ksat,crd,trd,po,pa) # unique folder indexer
        #outdirpath = '%s/%s'%(self.outpath,diridx)
        #os.makedirs(outdirpath) # create the output directory
        
        # run the simulation using the rhessys script
        cmd = 'sh /RHESSys/Como/scripts/rhessys_opt_ullr.sh %s %s %s %s %s %s %s %s %s %s %s %s %s %s'%(gw1,
        gw2,m,ksat,crd,trd,po,pa,self.start_date.year,self.start_date.month,self.start_date.day,self.end_date.year,
        self.end_date.month, self.end_date.day)
        
        print cmd
        
        subprocess.call(cmd,shell=True)
        
        # load the output
        streamflow = pd.read_table('/RHESSys/Como/out/opt/tmp/tmp_basin.daily',sep = ' ')['streamflow'].as_matrix() # load the output
        
        # load the streamflow data into the optimizer
        simulations = streamflow[self.clean_indexer==0] # clean out the nan values
        
        # remove the output directory
        #shutil.rmtree(outdirpath)
        return simulations


    def evaluation(self,evaldates=False):
        # function to return the evaluation data
        if evaldates==True:
            return self.evaldates
        
        if evaldates==False:
            return self.obs


    def objectivefunction(self,simulation,evaluation):
        # compute the objective function
        objectivefunction = spotpy.objectivefunctions.nashsutcliff(evaluation,simulation) # we want to maximize this function
        return objectivefunction


In [3]:
spotpy_setup = spotpy_setup()

In [4]:
sampler = spotpy.algorithms.sceua(spotpy_setup, dbname='testSCEUA', dbformat='csv')

In [5]:
sampler.sample(500, ngs=10, kstop = 30, pcento=0.005, peps=0.001)

sh /RHESSys/Como/scripts/rhessys_opt_ullr.sh 0.312104547732 0.510500220163 66.7107326536 113.701723178 6.30265333548 1.93949856619 20.6694892481 16.6388332962 2006 10 1 2009 10 5


/Users/barnhatb/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:70: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 1100 but corresponding boolean dimension is 731


0 of 500 (best like=-4.36709)
sh /RHESSys/Como/scripts/rhessys_opt_ullr.sh 0.121001628934 0.306080536163 81.2865141647 86.0331996488 8.71138443028 1.98223425464 6.09909339308 7.33063088533 2006 10 1 2009 10 5
1 of 500 (best like=-4.36709)
sh /RHESSys/Como/scripts/rhessys_opt_ullr.sh 0.346889248242 0.117811648616 91.0623101467 83.3593266731 4.0655250828 0.697804800795 18.1786201234 2.15515417997 2006 10 1 2009 10 5
2 of 500 (best like=-3.86351)
sh /RHESSys/Como/scripts/rhessys_opt_ullr.sh 0.38143854597 0.698011613773 71.2200624513 47.7171193403 7.63552540308 1.69990281991 22.3228790047 16.1116764083 2006 10 1 2009 10 5
3 of 500 (best like=-3.86351)
sh /RHESSys/Como/scripts/rhessys_opt_ullr.sh 0.168561179546 0.451738014183 71.4052551669 6.46400618943 6.4738949387 0.563076713684 12.790623909 5.98005922052 2006 10 1 2009 10 5
4 of 500 (best like=-3.86351)
sh /RHESSys/Como/scripts/rhessys_opt_ullr.sh 0.399946692723 0.191689537788 28.6756993724 56.7440611576 9.02902700208 0.290400719468 16.9

KeyboardInterrupt: 